# Program to scrape lilmiquela Instagram Photos and extract comments

### Import Libraries  

In [47]:
import pandas as pd
import numpy as np
import time
import re
import csv
from selenium import webdriver
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common import exceptions 
import requests
from bs4 import BeautifulSoup

In [48]:
def close_browser(browser):
    browser.close()
    return

### Get the summary of followers and posts count

In [49]:
# start a driver for a web browser:

browser  = webdriver.Chrome()
url = "https://www.instagram.com/lilmiquela/?hl=en"
browser.get(url)
browser.wait = WebDriverWait(browser, 5)

In [50]:
handle = "jasonyippee"

page = urlopen(url+handle).read()
data = bs(page, features="html.parser")

string = data.find("meta", property="og:description")['content']
print(string)

1.6m Followers, 2,012 Following, 531 Posts - See Instagram photos and videos from Miquela (@lilmiquela)


## Extract the Screens

In [51]:
links = []
ids   = []
likes = []

for x in range(1,10):
    
    body  = browser.find_element_by_tag_name('body')
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)                  
    
    source =  browser.page_source
    data = bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
   
    for link in script.findAll('a'):
        if re.match("/p", link.get('href')):
            links.append('https://www.instagram.com'+link.get('href'))
        likes.append
    
len(links)

366

In [52]:
a=pd.DataFrame(links)
a.columns = ['URL']
links = a
links.head(5)

,URL
0,https://www.instagram.com/p/BzZXzv8nIm2/
1,https://www.instagram.com/p/BzWDgavHW4V/
2,https://www.instagram.com/p/BzUCF10nxGb/
3,https://www.instagram.com/p/BzQ8TxjHjan/
4,https://www.instagram.com/p/BzOZRrxHBFJ/


In [53]:
close_browser(browser)

## Function to capture the emotions and remove from the scrapped comment texts

In [17]:
import emoji
def give_emoji_free_text(text):
    allchars = [str for str in text.decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

## Get the user comments for each screen


In [ ]:
path                = '/html[1]/body[1]/span[1]/section[1]/main[1]/div[1]/div[1]/article[1]/div[2]/div[1]/ul[1]/li[1]/div[1]/button[1]/span[1]'
data                = pd.DataFrame
users_list          = []
users_comment_list  = []
texts_list          = []
screen_list         = []
comments_Count_list = []
likes_Count_list    = []
i = j =0
    
for i in range(len(links)):
    
    browser      = webdriver.Chrome() 
    browser.wait = WebDriverWait(browser, 5)
    url          = links.URL[i]
    browser.get(url)
    print('Screen ',i, 'loaded')
    ###############################################
    #  get Likes and comment count for each screen
    ###############################################
    r              = requests.get(url)
    soup           = BeautifulSoup(r.content)
    sentence       = soup.find('meta', {'name': 'description'})['content']
    start_Like     = sentence.find('Likes')
    likes_Count    = sentence[0:start_Like].strip()
    start_Comment  = sentence.find('Comments')  
    comments_Count = (sentence[(len(sentence[0:start_Like]) +start_Like + 2):start_Comment]).strip()
    browser.wait   = WebDriverWait(browser, 5)
    body           = browser.find_element_by_tag_name('body') 
     
    try:
        if browser.find_element_by_xpath(path):
            browser.find_element_by_xpath(path).click()
        else:
            print('first page condition failed')
    except:
        pass

    #####################
    # Load next 10 pages 
    #####################
    
    print('Now loading page',i,'next 10 pages.....')
    LoadMore = 1
    
    try:
        while (LoadMore <=10):
            time.sleep(1)
            if browser.find_element_by_xpath(path):
                browser.find_element_by_xpath(path).click()
            LoadMore += 1
    except:
        print('next page condition failed')
        #break
        LoadMore += 1
    ###############
    # Get Comments
    ##############
    
    count = 0
    try:
        users = browser.find_elements_by_class_name('_6lAjh')
        for user in users:
            users_list.append(user.text)
            count += 1
    except exceptions.StaleElementReferenceException:
        pass

    k=len(users)
    j=0
    
    #################################
    # Get User and Post Text details
    #################################
    
    try:
        texts = browser.find_elements_by_xpath("//div[@class='C4VMK']/span")
        for txt in texts:
            if(j<count):
                users_comment_list.append(users_list[j][1])
                temp = txt.text.split(users_list[j][1])
                temp = re.sub(r"[^a-zA-Z0-9]", " ", str(temp))
                temp = give_emoji_free_text(temp.encode('utf8'))
                texts_list.append(temp)
                screen_list.append(url)
                comments_Count_list.append(comments_Count)
                likes_Count_list.append(likes_Count)
                j += 1
    except exceptions.StaleElementReferenceException:
        pass
        
    close_browser(browser)
    i += 1

data  = pd.DataFrame({'Screen' : screen_list, 'Likes' : likes_Count_list, 'Comments' : comments_Count_list,  'User' : users_list, 'Text' : texts_list})

   

Screen  0 loaded
Now loading page 0 next 10 pages.....
next page condition failed
Screen  1 loaded
Now loading page 1 next 10 pages.....
next page condition failed
Screen  2 loaded
Now loading page 2 next 10 pages.....
next page condition failed
Screen  3 loaded
Now loading page 3 next 10 pages.....
Screen  4 loaded
Now loading page 4 next 10 pages.....
next page condition failed
Screen  5 loaded
Now loading page 5 next 10 pages.....
next page condition failed
Screen  6 loaded
Now loading page 6 next 10 pages.....
next page condition failed
Screen  7 loaded
Now loading page 7 next 10 pages.....
next page condition failed
Screen  8 loaded
Now loading page 8 next 10 pages.....
next page condition failed
Screen  9 loaded
Now loading page 9 next 10 pages.....
Screen  10 loaded
Now loading page 10 next 10 pages.....
next page condition failed
Screen  11 loaded
Now loading page 11 next 10 pages.....
next page condition failed
Screen  12 loaded
Now loading page 12 next 10 pages.....
next page

In [26]:
data  = pd.DataFrame({'Screen' : screen_list, 'Likes' : likes_Count_list, 'Comments' : comments_Count_list,  'User' : users_list, 'Text' : texts_list})

In [27]:
data.head(10)

,Screen,Likes,Comments,User,Text
0,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,lilmiquela\nVerified,We harmon z ng
1,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,justmysecondaccount_yash,Oh god my q een is back really soon i can feel it
2,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,brickandmortar,YES THE CAPTION HAHAHA
3,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,ebks3speranta,This computer is still a thing
4,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,tylerxnathaniel,
5,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,dooweamoir.7,This is funny
6,https://www.instagram.com/p/Bz3okgPn4xc/,24.6k,40,actualfrogg,um hi I fu ken love you and we is harmonizing
7,https://www.instagram.com/p/Bz1O3n6Hm_K/,38.9k,79,lilmiquela\nVerified,soto gang and I s love connect on was soooo st...
8,https://www.instagram.com/p/Bz1O3n6Hm_K/,38.9k,79,iiinky420,jhendrick k n check ig
9,https://www.instagram.com/p/Bz1O3n6Hm_K/,38.9k,79,fuegoicecube,delete this account it does mo e ha m than goo...


In [28]:
data.to_csv('lilmiquela_Instagram_Comments.csv')  